<a href="https://colab.research.google.com/github/Muun-Muun/clusterd_sattelites_optimization/blob/main/CWCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 계산 클래스: CW 동역학 및 최적화 관련
import numpy as np
from scipy.linalg import expm, eig
from numpy.linalg import norm


class CWCalculator:
    def __init__(self, n):
        self.n = n
        self.A = np.array([
            [0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 1],
            [3 * n ** 2, 0, 0, 0, 2 * n, 0],
            [0, 0, 0, -2 * n, 0, 0],
            [0, 0, -n ** 2, 0, 0, 0]
        ])
        self.B = np.vstack([np.zeros((3, 3)), np.eye(3)])

    def compute_gamma(self, dt, steps=100):
        tau = np.linspace(0, dt, steps)
        integrand_sum = np.zeros((self.A.shape[0], self.B.shape[1]))
        for t in dt - tau:
            integrand_sum += expm(self.A * t) @ self.B
        return (dt / steps) * integrand_sum

    def cw_closed_form(self, t, x0):
        x0_, y0_, z0_, vx0_, vy0_, vz0_ = x0
        nt = self.n * t
        x = (4 - 3 * np.cos(nt)) * x0_ + np.sin(nt) / self.n * vx0_ + 2 * (1 - np.cos(nt)) / self.n * vy0_
        y = 6 * (np.sin(nt) - nt) * x0_ + y0_ - 2 * (1 - np.cos(nt)) / self.n * vx0_ + (4 * np.sin(nt) - 3 * nt) / self.n * vy0_
        z = z0_ * np.cos(nt) + vz0_ / self.n * np.sin(nt)
        return np.vstack((x, y, z)).T

    def propagate(self, x0, t_span=None, control=False, t_impulses=None, delta_vs=None, steps_per_segment=100):
        xk = x0.copy()
        trajectory = []
        times = []

        if not control:
            assert t_span is not None, "t_span must be provided if control=False"
            t = np.linspace(0, t_span, steps_per_segment)
            traj = self.cw_closed_form(t, xk)
            return traj, t
        else:
            assert t_impulses is not None and delta_vs is not None
            t_start = 0.0
            for i in range(len(t_impulses)):
                dt = t_impulses[i]
                t_segment = np.linspace(0, dt, steps_per_segment)
                traj_segment = self.cw_closed_form(t_segment, xk)
                trajectory.append(traj_segment)
                times.append(t_segment + t_start)
                t_start += dt
                Phi_k = expm(self.A * dt)
                Gamma_k = self.compute_gamma(dt)
                xk = Phi_k @ xk + Gamma_k @ delta_vs[i]
            return np.vstack(trajectory), np.hstack(times)

    def get_max_impact_delta_v(self, A, delta_v_max):
        ATA = A.T @ A
        eigvals, eigvecs = eig(ATA)
        v_opt = eigvecs[:, np.argmax(eigvals)]
        return v_opt / norm(v_opt) * delta_v_max

    def get_min_pc_delta_v(self, cov, delta_v_max):
        eigvals, eigvecs = eig(cov)
        v_dir = eigvecs[:, np.argmin(eigvals)]
        v_dir = v_dir / norm(v_dir) * delta_v_max
        return np.array([v_dir[0], v_dir[1], 0.0])
